In [1]:
%env LD_LIBRARY_PATH= / home / zach / anaconda3 / envs / research / lib /
%env TF_GPU_ALLOCATOR=cuda_malloc_async

env: LD_LIBRARY_PATH=/ home / zach / anaconda3 / envs / research / lib /
env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from tensorflow import keras
import os
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_addons as tfa

f1 = tfa.metrics.F1Score(num_classes=1, average=None)

tf.get_logger().setLevel('ERROR')

print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.9.0
Num GPUs Available:  1


2022-05-29 13:45:24.893585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 13:45:24.897049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 13:45:24.897357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 13:45:24.897823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
## Data Import and Cleaning
train = pd.read_csv('data/hateval2019_en_train.csv')
test = pd.read_csv('data/hateval2019_en_test.csv')
val = pd.read_csv('data/hateval2019_en_dev.csv')

train = train.drop(['TR', 'AG'], 1)
test = test.drop(['TR', 'AG'], 1)
val = val.drop(['TR', 'AG'], 1)
train.head()

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Flatten, concatenate, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L1, L2, l1_l2
import io


def normalize_tweet(text):
    """
    Removes hashtags, @s, links, and punctuation
    :param text:Text to be cleaned
    :return: text with mentions, hashtages, and urls removes
    """
    processed_text = text.lower()
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|t\.)\S+", "", processed_text)
    processed_text = re.sub(r"(?:\.|,|\?|-)", " ", processed_text)
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|\.com)", "", processed_text)
    processed_text = re.sub(r'[^\w\s]', '', processed_text)
    processed_text = " ".join(processed_text.split())
    return processed_text


def x_y_split(data):
    """splits and X and y from dataframe

    Args:
        data:dataframe to split from

    Returns:
        tuple:X data, y data
    """
    X = data['text']
    X = X.apply(normalize_tweet)
    y = data['HS']
    return X, y


## Split sequences into train, validation, and test sets
#Split x and ys
x_train, y_train = x_y_split(train)
print(x_train)
#x_train.to_csv('data/x_train.csv')
#y_train.to_csv('data/y_train.csv')
#Split x and ys
x_test, y_test = x_y_split(test)
print(x_test)
#x_test.to_csv('data/x_test.csv')
#y_test.to_csv('data/y_test.csv')
#Split x and ys
x_val, y_val = x_y_split(val)
print(x_val)
#x_val.to_csv('data/x_val.csv')
#y_val.to_csv('data/y_val.csv')

x_graph_train = pd.read_csv('data/extracted_annotations_train.csv')
x_graph_val = pd.read_csv('data/extracted_annotations_val.csv')
x_graph_test = pd.read_csv('data/extracted_annotations_test.csv')

x_graph_train = x_graph_train['annotations']
x_graph_val = x_graph_val['annotations']
x_graph_test = x_graph_test['annotations']

x_graph_train = x_graph_train.apply(str)
x_graph_val = x_graph_val.apply(str)
x_graph_test = x_graph_test.apply(str)

/tmp/ipykernel_11376/3597924913.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(['TR', 'AG'], 1)
/tmp/ipykernel_11376/3597924913.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop(['TR', 'AG'], 1)
/tmp/ipykernel_11376/3597924913.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  val = val.drop(['TR', 'AG'], 1)


0       hurray saving us in so many ways lockthemup bu...
1       why would young fighting age men be the vast m...
2       illegals dump their kids at the border like ro...
3       ny times nearly all white states pose an array...
4       orban in brussels european leaders are ignorin...
                              ...                        
8995                  i am proud to be a hysterical woman
8996    hollywood is complicit in the rape and sexual ...
8997    what a fucking cunt i hate seeing kids getting...
8998                                hysterical woman like
8999    nearly every woman i know has metoo in their f...
Name: text, Length: 9000, dtype: object
0       oh i could have gone on about taxes since the ...
1       several of the wild fires in california and co...
2       my question is how do you resettle a refugee a...
3       europe youve got a problem we must hurry and b...
4       this is outrageous stopillegalimmigration meri...
                              ..

In [4]:
x_train

0       hurray saving us in so many ways lockthemup bu...
1       why would young fighting age men be the vast m...
2       illegals dump their kids at the border like ro...
3       ny times nearly all white states pose an array...
4       orban in brussels european leaders are ignorin...
                              ...                        
8995                  i am proud to be a hysterical woman
8996    hollywood is complicit in the rape and sexual ...
8997    what a fucking cunt i hate seeing kids getting...
8998                                hysterical woman like
8999    nearly every woman i know has metoo in their f...
Name: text, Length: 9000, dtype: object

In [5]:
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model

tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'


def build_classifier_model(do, trainable=False):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=trainable, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    text_input2 = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_graph')
    preprocessing_layer2 = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_graph')
    encoder_inputs2 = preprocessing_layer2(text_input2)
    encoder2 = hub.KerasLayer(tfhub_handle_encoder, trainable=trainable, name='BERT_encoder_graph')
    outputs2 = encoder2(encoder_inputs2)
    net2 = outputs2['pooled_output']
    combined = concatenate([net,net2])
    x = Dropout(do)(combined)
    x = Reshape((1,1536))(x)
    x = LSTM(512,dropout=do)(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(do)(x)
    x = Dense(256,activation='relu')(x)
    x = Dense(1,activation='sigmoid')(x)

    return tf.keras.Model([text_input,text_input2], x)


loss = tf.keras.losses.BinaryCrossentropy()


In [ ]:
learning_rates = [5e-5, 1e-4, 3e-5, 3e-5]
#learning_rates = [5e-5]
learning_rates_text = ['5e-5', '1e-4', '3e-5', '3e-5']
batch_sizes = [8]
dropout = [.5,.75,.25,0.0,.1]
#dropout = [.1]
results = {}
results_acc = {}
best_epochs = {}
for do in dropout:
    for bs in batch_sizes:
        for lr in range(len(learning_rates)):
            np.random.seed(42)
            tf.random.set_seed(42)
            string = 'lr{lr}_bs{bs}_do{do}'.format(lr=learning_rates_text[lr], bs=bs, do=do)

            bert = build_classifier_model(do, trainable=True)
            bert.compile(loss=loss, optimizer=Adam(learning_rate=learning_rates[lr]), metrics=[f1, 'accuracy'])
            history = bert.fit(x=[x_train,x_graph_train],
                               y=y_train, batch_size=8,
                               validation_data=([x_val,x_graph_val], y_val),
                               epochs=4)

            max_f1 = max(history.history['val_f1_score'])
            max_acc = max(history.history['val_accuracy'])
            val_acc_per_epoch = history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max_acc) + 1
            results[string] = max_f1
            results_acc[string] = max_acc
            best_epochs[string] = best_epoch

Epoch 1/4


2022-05-29 13:45:52.510128: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


 396/1125 [=========>....................] - ETA: 3:46 - loss: 0.6868 - f1_score: 0.5992 - accuracy: 0.5619

In [ ]:
results_acc

In [ ]:
bert.summary()